In [5]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import tensorflow as tf

##### SETUP

In [6]:
#@title Install required libraries

!pip install keras~=3.8.0 \
  matplotlib~=3.10.0 \
  numpy~=2.0.0 \
  pandas~=2.2.0 \
  tensorflow~=2.18.0

print('\n\nAll requirements successfully installed.')



All requirements successfully installed.


In [7]:
#@title Code - Load dependencies

#general
import io

# data
import numpy as numpy
import pandas as panda

# machine learning
import keras

# data visualization
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn
import tensorflow as tf

import kagglehub

In [8]:
#@title Mempersiapkan dataset
path = kagglehub.dataset_download("fuadkahfi/dataset-tanah")+"/Tanah"
print(path)

batch = 80 #@param
p = 256 #@param
l = 256 #@param
split = 0.2 # @param {"type":"number"}

training_dataset = tf.keras.utils.image_dataset_from_directory(
    path,
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=batch,

    validation_split=split, #80% dataset yang akan di latih
    subset="training",
    seed=1,

    image_size=(p, l),
    shuffle=True
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    path,
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=batch,

    validation_split=split, #20% dataset yang akan diuji
    subset="validation",
    seed=1,

    image_size=(p, l),
    shuffle=True
)


/kaggle/input/dataset-tanah/Tanah
Found 880 files belonging to 8 classes.
Using 704 files for training.
Found 880 files belonging to 8 classes.
Using 176 files for validation.


In [9]:
#@title Membuat model

jenis = len(training_dataset.class_names)
print(jenis)
droput = 0.1 #@param

augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomZoom(0.025),
    tf.keras.layers.RandomContrast(0.01),
    tf.keras.layers.RandomBrightness(0.01),
])

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(p, l, 3)),
    tf.keras.layers.Rescaling(1./255),
    augmentation,

    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Dropout(droput),
    tf.keras.layers.GlobalAveragePooling2D(), # Add this layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(droput),
    tf.keras.layers.Dense(jenis, activation='softmax')
])

8


In [10]:
#@title Melatih model dan validaasi model

rate = 0.001 # @param {"type":"number"}
epoch = 30 # @param {"type":"integer"}

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=rate),
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

history = model.fit(
    training_dataset,
    validation_data=validation_dataset,
    epochs=epoch
)

Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 266s 28s/step - accuracy: 0.1237 - loss: 2.0819 - val_accuracy: 0.2273 - val_loss: 2.0630
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 243s 27s/step - accuracy: 0.1848 - loss: 2.0618 - val_accuracy: 0.2386 - val_loss: 2.0008
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 266s 27s/step - accuracy: 0.1985 - loss: 2.0032 - val_accuracy: 0.2443 - val_loss: 1.8818
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 264s 27s/step - accuracy: 0.1954 - loss: 1.9645 - val_accuracy: 0.2330 - val_loss: 1.8495
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 252s 28s/step - accuracy: 0.1848 - loss: 1.9325 - val_accuracy: 0.2045 - val_loss: 1.8417
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 258s 27s/step - accuracy: 0.2186 - loss: 1.9441 - val_accuracy: 0.2841 - val_loss: 1.7470
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 259s 27s/step - accuracy: 0.2489 - loss: 1.8446 - val_accuracy: 0.2386 - val_loss: 1.7172
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 270s 28s/step - accuracy: 0.2305 - loss: 1.8962 - val_accuracy: 0.2443 - val_loss:

KeyboardInterrupt: 

In [ ]:
#@title Statistik
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

# Image classification with TensorFlow Lite Model Maker with TensorFlow 2.18.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/lite/tutorials/model_maker_image_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/tutorials/model_maker_image_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/tutorials/model_maker_image_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/tutorials/model_maker_image_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This notebook has been moved [here](https://www.tensorflow.org/lite/tutorials/model_maker_image_classification).

In [ ]:

print(tf.__version__)

https://colab.research.google.com/github/TrainingBear/OPSI-Project-AndroidApplication/blob/master/tensorflow_examples/lite/model_maker/demo/image_classification.ipynb#scrollTo=w7AdazbUlvDJ